# BERT

The second model used in this thesis is the BERT model. The reasoning for this choice is that the BERT model is bit different than other LLMs in a sense that while being pre-trained, the BERT model is trained bidirectionaly. This means that the model has both past and future context, unlike other LLMs that only have past context. The model also offers a middle ground in terms of size as it sits at around 110 million parameters, which is significantly larger than the BiLSTM, but significantly smaller than the Llama model which has 3 billion parameters. 

## Imports

In [ ]:
!pip install optuna
!pip uninstall transformers -y
!pip install transformers
!pip install python-dotenv
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 7.4 MB/s eta 0:00:00
Found existing installation: transformers 4.48.3
Uninstalling transformers-4.48.3:
  Successfully uninstalled transformers-4.48.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 77.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.2 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    BitsAndBytesConfig,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
    EarlyStoppingCallback
)
from peft import(
    LoraConfig,
    prepare_model_for_kbit_training,
    get_peft_model,
    PeftModel
)
from dotenv import load_dotenv
from datasets import Dataset
from sklearn.metrics import accuracy_score, f1_score
from huggingface_hub import login
#Insert hugging face api key below
#login('')

load_dotenv()

## Model Building

In this section, the best hyperparameters for the model are first chosen, then the model is fully finetuned on the dataset.

### Model and Tokenizer

In [ ]:
id2label = {0: 'Depression', 1: 'Normal', 2: 'Anxiety', 3: 'Suicidal'}
label2id = {'Depression': 0, 'Normal': 1,'Anxiety': 2, 'Suicidal':3 }
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModelForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=4,
 id2label=id2label, label2id=label2id

)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Hyperparameter tuning

#### Data Pre-processing

The pre-processing steps for the LLMs are the same and are very simple. The comprise of turning the dataframes into datasets so huggingface can handle them, and then using the pre-trained model tokenizer to tokenize the datasets while applying truncation.

In [ ]:
id2label = {0: 'Depression', 1: 'Normal', 2: 'Anxiety', 3: 'Suicidal'}
label2id = {'Depression': 0, 'Normal': 1,'Anxiety': 2, 'Suicidal':3 }
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["italian text"], truncation=True)


In [ ]:
X_train = pd.read_csv('/content/drive/MyDrive/Italian thesis/Training dataset/train_sample.csv')
X_val = pd.read_csv('/content/drive/MyDrive/Italian thesis/Training dataset/val_sample.csv')

# Merge them by concatenation
print(f"Training set size: {len(X_train)}")
print(f"Validation set size: {len(X_val)}")
#print(f"Test set size: {len(X_test)}")
def tokenize(example):
    return tokenizer(example["italian text"], padding=True, truncation=True, max_length=tokenizer.model_max_length)
train = Dataset.from_pandas(X_train)
#test = Dataset.from_pandas(X_test)
validation = Dataset.from_pandas(X_val)
tokenized_train = train.map(tokenize, batched=True)
#tokenized_test=test.map(tokenize, batched=True)
tokenized_val=validation.map(tokenize, batched=True)
id2label = {0: 'Depression', 1: 'Normal', 2: 'Anxiety', 3: 'Suicidal'}
label2id = {'Depression': 0, 'Normal': 1,'Anxiety': 2, 'Suicidal':3}


Training set size: 3200
Validation set size: 600


Map:   0%|          | 0/3200 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

In [ ]:
tokenized_train = tokenized_train.remove_columns(["text", "italian text"])
tokenized_val = tokenized_val.remove_columns(["text", "italian text"])
#tokenized_test = tokenized_test.remove_columns(["text", "italian text"])  # If using test data


After pre-processing the dataset, the hyperparameter tuning process begins. The first step is to define the model_init function. Then the next step is to define the hyperparameter space that will be tested by the model. Next, the metrics for analyzing the performance of the different models is defined, which are Precision, Recall, Accuracy, and F1 Score. The tuning process then begins, with the aim of maximizing the F1 score. 

In [ ]:
def model_init():
  model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=4)
  tokenizer = AutoTokenizer.from_pretrained(
    'bert-base-uncased',
    add_prefix_space=True
)
  tokenizer.pad_token_id = tokenizer.eos_token_id
  tokenizer.pad_token = tokenizer.eos_token

  model.config.pad_token_id = tokenizer.pad_token_id
  return model

In [ ]:
def optuna_hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-5, 5e-5, log=True),
        "gradient_accumulation_steps": trial.suggest_categorical("gradient_accumulation_steps", [1, 2, 4, 8]),  # Accumulation steps
        "lr_scheduler_type": trial.suggest_categorical("lr_scheduler_type", ["linear", "cosine", "constant", "cosine_with_restarts"]),  # Scheduler
        "warmup_ratio": trial.suggest_float("warmup_ratio", 0.0, 0.1),  # Warmup ratio
        "weight_decay": trial.suggest_float("weight_decay", 0.01, 0.1),
    }


In [ ]:
def compute_objective(metrics):
    print(metrics)
    return metrics["eval_f1_macro"]

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import numpy as np
import torch
from datasets import Dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer,   DataCollatorWithPadding,Trainer, TrainingArguments

from tqdm import tqdm
from sklearn.metrics import accuracy_score,f1_score, confusion_matrix, classification_report
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score, precision_score, recall_score
import numpy as np
import pandas as pd

# Define your class labels
id2label = {0: 'Depression', 1: 'Normal', 2: 'Anxiety', 3: 'Suicidal'}
label2id = {v: k for k, v in id2label.items()}

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # Compute accuracy
    accuracy = accuracy_score(labels, predictions)

    # Compute F1 score (macro-average)
    f1 = f1_score(labels, predictions, average='macro')

    # Compute precision and recall (macro-average)
    precision = precision_score(labels, predictions, average='macro')
    recall = recall_score(labels, predictions, average='macro')

    # Compute confusion matrix
    cm = confusion_matrix(labels, predictions)

    # Convert confusion matrix from class IDs to labels
    cm_labels = np.array([id2label[i] for i in range(len(id2label))])
    cm_with_labels = pd.DataFrame(cm, index=cm_labels, columns=cm_labels)

    # Generate the classification report with labels
    class_report = classification_report(labels, predictions, target_names=[id2label[i] for i in range(len(id2label))])

    # Print confusion matrix and classification report
    print("Confusion Matrix:")
    print(cm_with_labels)
    print("\nClassification Report:")
    print(class_report)

    return {
        'accuracy': accuracy,
        'f1_macro': f1,
        'precision_macro': precision,
        'recall_macro': recall,
    }


In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir="BERT-hyperparameter-search",
    num_train_epochs=4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    metric_for_best_model="f1_macro",
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
)
trainer = Trainer(
    model=None,
    model_init=model_init,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    compute_metrics=compute_metrics,
    data_collator=data_collator)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
best_trial = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=optuna_hp_space,
    n_trials=30,
    compute_objective=compute_objective,
)

[I 2025-02-26 20:31:19,364] A new study created in memory with name: no-name-4155925a-b8c6-43a0-8a7f-cd6b90c5c2fb
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: masabbah97 (aml_group) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


...

[I 2025-02-26 21:41:44,634] Trial 28 pruned. 
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


eval/accuracy,▁
eval/f1_macro,▁
eval/loss,▁
eval/precision_macro,▁
eval/recall_macro,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁
train/global_step,▁
eval/accuracy,0.56


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,Precision Macro,Recall Macro
1,No log,1.090234,0.555000,0.525553,0.561591,0.555000


Confusion Matrix:
            Depression  Normal  Anxiety  Suicidal
Depression          55      18       53        24
Normal               1     137       11         1
Anxiety             11      31      103         5
Suicidal            26      30       56        38

Classification Report:
              precision    recall  f1-score   support

  Depression       0.59      0.37      0.45       150
      Normal       0.63      0.91      0.75       150
     Anxiety       0.46      0.69      0.55       150
    Suicidal       0.56      0.25      0.35       150

    accuracy                           0.56       600
   macro avg       0.56      0.56      0.53       600
weighted avg       0.56      0.56      0.53       600

{'eval_loss': 1.0902342796325684, 'eval_accuracy': 0.555, 'eval_f1_macro': 0.5255528626217767, 'eval_precision_macro': 0.5615910115512847, 'eval_recall_macro': 0.555}


[I 2025-02-26 21:42:49,304] Trial 29 pruned. 


In [ ]:
print("Best Trial:")
print("Params:", best_trial.hyperparameters)

Best Trial:
Params: {'learning_rate': 3.543919690225377e-05, 'gradient_accumulation_steps': 1, 'lr_scheduler_type': 'linear', 'warmup_ratio': 0.09785956688181825, 'weight_decay': 0.028924075405665158}


In [ ]:
import os

# Change the current working directory
os.chdir("/content/drive/MyDrive/Italian thesis/")

# Verify the current working directory
print("Current Working Directory:", os.getcwd())

Current Working Directory: /content/drive/MyDrive/Italian thesis


In [ ]:
import json
json_file_path = "BERT_hyperparameters.json"

# Write hyperparameters to a JSON file
with open(json_file_path, 'w') as json_file:
    json.dump(best_trial.hyperparameters, json_file, indent=4)

print(f"Hyperparameters saved to {json_file_path}")

Hyperparameters saved to BERT_hyperparameters.json


### Finetuning

After Hyperparameter tuning the model, the next step is to use the best performing hyperparameters to fully finetune the model on the mental health dataset. 

#### Data Pre-processing

The same steps as the hyperparameter tuning process are done here.

In [ ]:
X_train = pd.read_csv('/content/drive/MyDrive/Italian thesis/Training dataset/train.csv')
#X_train = X_train.drop(columns=["text"]).rename(columns={"italian text": "text"})
X_train = X_train.dropna()
X_val = pd.read_csv('/content/drive/MyDrive/Italian thesis/Training dataset/val.csv')

#X_val = X_val.drop(columns=["text"]).rename(columns={"italian text": "text"})
X_test = pd.read_csv('/content/drive/MyDrive/Italian thesis/Training dataset/test.csv')
#X_test = X_test.drop(columns=["text"]).rename(columns={"italian text": "text"})
print(f"Training set size: {len(X_train)}")
print(f"Validation set size: {len(X_val)}")
print(f"Test set size: {len(X_test)}")

Training set size: 11760
Validation set size: 1470
Test set size: 1470


In [ ]:
X_train.head()

,text,label,italian text
0,Im at the point of the semester where Im so ti...,3,Sono arrivato al punto del semestre in cui son...
1,I could do it I want to do it I have what I ne...,3,Potrei farlo Voglio farlo Ho ciò che mi serve ...
2,I cannot imagine anyone wanting to be in a rel...,3,Non riesco a immaginare qualcuno che voglia av...
3,How I am supposed to live if I cannot accept t...,3,Come dovrei vivere se non riesco ad accettare ...
4,maybe i know but how could you know,1,forse lo so ma come potresti saperlo?


In [ ]:
train = Dataset.from_pandas(X_train)
test = Dataset.from_pandas(X_test)
validation = Dataset.from_pandas(X_val)

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["italian text"], truncation=True)


In [ ]:
tokenized_train = train.map(preprocess_function, batched=True)
tokenized_test = test.map(preprocess_function, batched=True)
tokenized_val = validation.map(preprocess_function, batched=True)


Map:   0%|          | 0/11760 [00:00<?, ? examples/s]

Map:   0%|          | 0/1470 [00:00<?, ? examples/s]

Map:   0%|          | 0/1470 [00:00<?, ? examples/s]

In [ ]:
tokenized_train = tokenized_train.remove_columns(["text", "italian text"])
tokenized_val = tokenized_val.remove_columns(["text", "italian text"])
tokenized_test = tokenized_test.remove_columns(["text", "italian text"])  # If using test data


In [ ]:
print(tokenized_test.column_names)


['label', 'input_ids', 'token_type_ids', 'attention_mask']


In [ ]:
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=2,
    early_stopping_threshold=0.01
)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

#### Model Training

For the Training stage, the first step is to define the metrics for the model performance. Then the next step is to define the trainer and its arguments using the ones obtained from the hyperparameter tuning process. The last step is of course to train the model.

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import numpy as np
import torch
from datasets import Dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer,   DataCollatorWithPadding,Trainer, TrainingArguments

from tqdm import tqdm
from sklearn.metrics import accuracy_score,f1_score, confusion_matrix, classification_report
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score, precision_score, recall_score
import numpy as np
import pandas as pd

# Define your class labels
id2label = {0: 'Depression', 1: 'Normal', 2: 'Anxiety', 3: 'Suicidal'}
label2id = {v: k for k, v in id2label.items()}

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # Compute accuracy
    accuracy = accuracy_score(labels, predictions)

    # Compute F1 score (macro-average)
    f1 = f1_score(labels, predictions, average='macro')

    # Compute precision and recall (macro-average)
    precision = precision_score(labels, predictions, average='macro')
    recall = recall_score(labels, predictions, average='macro')

    # Compute confusion matrix
    cm = confusion_matrix(labels, predictions)

    # Convert confusion matrix from class IDs to labels
    cm_labels = np.array([id2label[i] for i in range(len(id2label))])
    cm_with_labels = pd.DataFrame(cm, index=cm_labels, columns=cm_labels)

    # Generate the classification report with labels
    class_report = classification_report(labels, predictions, target_names=[id2label[i] for i in range(len(id2label))])

    # Print confusion matrix and classification report
    print("Confusion Matrix:")
    print(cm_with_labels)
    print("\nClassification Report:")
    print(class_report)

    return {
        'accuracy': accuracy,
        'f1_macro': f1,
        'precision_macro': precision,
        'recall_macro': recall,
    }


In [ ]:
training_args = TrainingArguments(
    output_dir="bert-finetuning-italian",
    learning_rate=2.1612703354421325e-05,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay= 0.028924075405665158,
    gradient_accumulation_steps=1,
    lr_scheduler_type="cosine_with_restarts",
    warmup_ratio=0.051758482154894515,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping]
)

<ipython-input-13-67bf0fabcbff>:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
results = trainer.evaluate(eval_dataset=tokenized_test)
print('Results before finetuning:')
print('-'*15)

print(results)

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m

Confusion Matrix:
            Depression  Normal  Anxiety  Suicidal
Depression         110     278        0        12
Normal             118     152        0       130
Anxiety             60     194        0        16
Suicidal            96     292        0        12

Classification Report:
              precision    recall  f1-score   support

  Depression       0.29      0.28      0.28       400
      Normal       0.17      0.38      0.23       400
     Anxiety       0.00      0.00      0.00       270
    Suicidal       0.07      0.03      0.04       400

    accuracy                           0.19      1470
   macro avg       0.13      0.17      0.14      1470
weighted avg       0.14      0.19      0.15      1470



wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: masabbah97 (aml_group) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Results before finetuning:
---------------
{'eval_loss': 1.4045946598052979, 'eval_model_preparation_time': 0.0031, 'eval_accuracy': 0.18639455782312925, 'eval_f1_macro': 0.13843013689238293, 'eval_precision_macro': 0.13074635831406797, 'eval_recall_macro': 0.17125, 'eval_runtime': 6.2067, 'eval_samples_per_second': 236.841, 'eval_steps_per_second': 14.823}


In [ ]:
trainer.train()


Epoch,Training Loss,Validation Loss,Model Preparation Time,Accuracy,F1 Macro,Precision Macro,Recall Macro
1,1.092800,0.722230,0.003100,0.687755,0.693287,0.720715,0.697778
2,0.682700,0.613302,0.003100,0.738095,0.745604,0.753249,0.747940
3,0.473000,0.606387,0.003100,0.754422,0.763796,0.770798,0.760231
4,0.425900,0.710029,0.003100,0.746939,0.757815,0.763959,0.753356
5,0.318700,0.748144,0.003100,0.752381,0.758624,0.763078,0.758657


Confusion Matrix:
            Depression  Normal  Anxiety  Suicidal
Depression         148      11       25       216
Normal               5     334        8        53
Anxiety             20       9      219        22
Suicidal            57      25        8       310

Classification Report:
              precision    recall  f1-score   support

  Depression       0.64      0.37      0.47       400
      Normal       0.88      0.83      0.86       400
     Anxiety       0.84      0.81      0.83       270
    Suicidal       0.52      0.78      0.62       400

    accuracy                           0.69      1470
   macro avg       0.72      0.70      0.69      1470
weighted avg       0.71      0.69      0.68      1470

Confusion Matrix:
            Depression  Normal  Anxiety  Suicidal
Depression         286      12       29        73
Normal              22     358        7        13
Anxiety             25       9      232         4
Suicidal           161      24        6       209

Clas

TrainOutput(global_step=3675, training_loss=0.5508762043025218, metrics={'train_runtime': 783.2003, 'train_samples_per_second': 150.153, 'train_steps_per_second': 9.385, 'total_flos': 9762697238890752.0, 'train_loss': 0.5508762043025218, 'epoch': 5.0})

### Results

When looking at the results of BERT model, some similar patterns to the ones exhibited by the AB-BiLSTM model are seen, albeit with a better overall performance at an accuracy of 77% and an F1-score of 78%. Overall, the model has the most trouble with the ’Depression’ class with a low class specific F1-score of 63% and the same misclassification problem when it comes to this class and the ’Suicidal’ one. But overall, the model also performs well on both the ’Normal’ and the ’Anxiety’ classes as they are easier to distinguish than the other two.

In [ ]:
results = trainer.evaluate(eval_dataset=tokenized_test)
print('Results after finetuning:')
print('-'*15)
print(results)

Confusion Matrix:
            Depression  Normal  Anxiety  Suicidal
Depression         239       8       21       132
Normal              19     359        8        14
Anxiety             40       6      218         6
Suicidal            60      12        5       323

Classification Report:
              precision    recall  f1-score   support

  Depression       0.67      0.60      0.63       400
      Normal       0.93      0.90      0.91       400
     Anxiety       0.87      0.81      0.84       270
    Suicidal       0.68      0.81      0.74       400

    accuracy                           0.77      1470
   macro avg       0.79      0.78      0.78      1470
weighted avg       0.78      0.77      0.77      1470

Results after finetuning:
---------------
{'eval_loss': 0.5598059892654419, 'eval_model_preparation_time': 0.0031, 'eval_accuracy': 0.7748299319727892, 'eval_f1_macro': 0.7796978245295705, 'eval_precision_macro': 0.7862861657275065, 'eval_recall_macro': 0.7774768518518519,

In [ ]:
trainer.push_to_hub()

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

events.out.tfevents.1740662535.86163c9bd3c3.2313.1:   0%|          | 0.00/644 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

events.out.tfevents.1740661635.86163c9bd3c3.2313.0:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.37k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/msab97/bert-finetuning-italian/commit/45e2e9d654301bb7fe4fbdb9566bf8760dbc9a8c', commit_message='End of training', commit_description='', oid='45e2e9d654301bb7fe4fbdb9566bf8760dbc9a8c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/msab97/bert-finetuning-italian', endpoint='https://huggingface.co', repo_type='model', repo_id='msab97/bert-finetuning-italian'), pr_revision=None, pr_num=None)

## Predictions

In order to build the indicator, the model will be used to classify a different set of tweets. These tweets are sampled (3000 for each day) from a dataset that contains 15 million tweets from the first 5 months of 2020. The same pre-processing steps are applied to this new dataset, these tweets are then using as input and the final predictions as well as the confidence values are then stored.

In [ ]:


from peft import PeftModel
from transformers import AutoModelForSequenceClassification,BitsAndBytesConfig, AutoTokenizer,EarlyStoppingCallback,DataCollatorWithPadding
import torch
from huggingface_hub import login
import pandas as pd
from datasets import Dataset
import torch

#login('')


In [3]:
df = pd.read_csv('/content/drive/MyDrive/Italian thesis/Training dataset/italian_with_predictions_and_logits.csv')
df.head()

,testo,tweet_date,llama_logits,llama_prediction
0,l istruzione e la ricchezza posson essere sorg...,2020-01-31,"[-2.353515625, 4.1796875, -2.923828125, -2.949...",1
1,ce l abbiamo fatta tutti in pratica parco nord...,2020-01-31,"[0.2763671875, 3.181640625, -5.38671875, 0.232...",1
2,qualcuno poi mi spieghi tutto il credito che s...,2020-01-31,"[-0.0018262863159179688, 2.484375, -7.28125, -...",1
3,aforismi f duva mov stelle e chi lo rappresent...,2020-01-31,"[0.78662109375, 3.923828125, -7.43359375, 0.48...",1
4,di solito quando un prodotto ha bisogno di tan...,2020-01-31,"[-0.41552734375, 5.20703125, -7.1796875, -1.48...",1


In [4]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("msab97/bert-finetuning-italian")
model = AutoModelForSequenceClassification.from_pretrained("msab97/bert-finetuning-italian")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/925 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [5]:
def preprocess_function(examples):
    return tokenizer(examples["testo"], truncation=True)

In [6]:
data = Dataset.from_pandas(df)
tokenized_data = data.map(preprocess_function, batched=True)
tokenized_data = tokenized_data.remove_columns(["testo", "tweet_date"])


Map:   0%|          | 0/456000 [00:00<?, ? examples/s]

In [7]:
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=2,
    early_stopping_threshold=0.01
)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [8]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import numpy as np
import torch
from datasets import Dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer,   DataCollatorWithPadding,Trainer, TrainingArguments

from tqdm import tqdm  # Import tqdm for progress bar
from sklearn.metrics import accuracy_score,f1_score, confusion_matrix, classification_report
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score
import numpy as np

id2label = {0: 'Depression', 1: 'Normal', 2: 'Anxiety', 3: 'Suicidal'}
label2id = {v: k for k, v in id2label.items()}

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    accuracy = accuracy_score(labels, predictions)

    f1 = f1_score(labels, predictions, average='macro')

    cm = confusion_matrix(labels, predictions)

    cm_labels = np.array([id2label[i] for i in range(len(id2label))])
    cm_with_labels = pd.DataFrame(cm, index=cm_labels, columns=cm_labels)

    class_report = classification_report(labels, predictions, target_names=[id2label[i] for i in range(len(id2label))])

    print("Confusion Matrix:")
    print(cm_with_labels)
    print("\nClassification Report:")
    print(class_report)

    return {
        'accuracy': accuracy,
        'f1_macro': f1,
    }

In [9]:
training_args = TrainingArguments(
    output_dir="bert-finetuning-italian",
    per_device_eval_batch_size=128,
    logging_dir='./logs',
)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

In [ ]:
predictions = trainer.predict(tokenized_data)

logits = torch.tensor(predictions.predictions)

df['bert_logits'] = logits.tolist()

predicted_classes = torch.argmax(logits, dim=-1).numpy()
df['bert_prediction'] = predicted_classes


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: masabbah97 (aml_group) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [11]:
df.to_csv("/content/drive/MyDrive/Italian thesis/Training dataset/italian_with_predictions_and_logits.csv", index=False)

In [12]:
df.head()

,testo,tweet_date,llama_logits,llama_prediction,bert_logits,bert_prediction
0,l istruzione e la ricchezza posson essere sorg...,2020-01-31,"[-2.353515625, 4.1796875, -2.923828125, -2.949...",1,"[-0.4601691961288452, 3.44775390625, -1.441370...",1
1,ce l abbiamo fatta tutti in pratica parco nord...,2020-01-31,"[0.2763671875, 3.181640625, -5.38671875, 0.232...",1,"[-1.0215137004852295, 4.273667335510254, -1.83...",1
2,qualcuno poi mi spieghi tutto il credito che s...,2020-01-31,"[-0.0018262863159179688, 2.484375, -7.28125, -...",1,"[1.2525972127914429, 0.6674477458000183, -1.16...",0
3,aforismi f duva mov stelle e chi lo rappresent...,2020-01-31,"[0.78662109375, 3.923828125, -7.43359375, 0.48...",1,"[-0.9413514137268066, 4.013585567474365, -0.89...",1
4,di solito quando un prodotto ha bisogno di tan...,2020-01-31,"[-0.41552734375, 5.20703125, -7.1796875, -1.48...",1,"[-0.9259320497512817, 3.9972760677337646, -1.7...",1
